In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/antecedents.tsv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/consequents.tsv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/simplified.tsv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/antecedents/submit_results.csv /content/antecedent_results.csv
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/consequents/submit_results.csv /content/consequent_results.csv
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/simplified/submit_results.csv /content/simplified_results.csv

In [0]:
import pandas as pd

prefix = '/content/'

test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1].replace(r'[,;:#&()`\'\"]', '', regex=True)
})

# display 
test_df.head()

In [0]:
pip install pytorch_pretrained_bert

In [0]:
from pytorch_pretrained_bert import BertTokenizer,BertForMaskedLM
import torch
import pandas as pd
import math

bertMaskedLM = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
def get_score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=bertMaskedLM(tensor_input)
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions.squeeze(),tensor_input.squeeze()).data 
    return math.exp(loss)

In [0]:
import pandas as pd

prefix = '/content/'

a_df = pd.read_csv(prefix + 'antecedents.tsv', sep='\t', delimiter=None, header=None)
a_df = pd.DataFrame({
    'id': a_df[0],
    'labels':a_df[1],
    'text': a_df[2].replace(r'[,;:#&()`\'\"]', '', regex=True)
})
c_df = pd.read_csv(prefix + 'consequents.tsv', sep='\t', delimiter=None, header=None)
c_df = pd.DataFrame({
    'id': c_df[0],
    'labels':c_df[1],
    'text': c_df[2].replace(r'[,;:#&()`\'\"]', '', regex=True)
})
s_df = pd.read_csv(prefix + 'simplified.tsv', sep='\t', delimiter=None, header=None)
s_df = pd.DataFrame({
    'id': s_df[0],
    'labels':s_df[1],
    'text': s_df[2].replace(r'[,;:#&()`\'\"]', '', regex=True)
})

rc_df = pd.read_csv(prefix + 'consequent_results.csv', header=None)
rc_df = pd.DataFrame({
    'id': rc_df[0],
    'labels':rc_df[1]
})
ra_df = pd.read_csv(prefix + 'antecedent_results.csv', header=None)
ra_df = pd.DataFrame({
    'id': ra_df[0],
    'labels':ra_df[1]
})
rs_df = pd.read_csv(prefix + 'simplified_results.csv', header=None)
rs_df = pd.DataFrame({
    'id': rs_df[0],
    'labels':rs_df[1]
})

s_df.head()

In [0]:
from tqdm import tqdm
from time import sleep

submit_df= pd.DataFrame(columns=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid'])
t=0
for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]
  id = test_df['id'][ind]
  #print(id, original)
  
  modela = ra_df.loc[ra_df.id.str.contains(id)]
  modelc = rc_df.loc[rc_df.id.str.contains(id)]
  models = rs_df.loc[rs_df.id.str.contains(id)]

  antecedent = str()
  consequent = str()
  text= str()
  alab ='0' # assume no
  for a in modela.index:
    amid = modela['id'][a]  
    alab = modela['labels'][a]
    if alab == 1:
      #print(amid)
      antecd = a_df.loc[a_df.id.str.contains(amid)]
      assert antecd['id'].count() == 1 # 1-to-1
      for i in antecd.index:
        atext = antecd['text'][i]
        antecedent = atext
        text = atext

  clab = '0'
  for c in modelc.index:
    cmid = modelc['id'][c]  
    clab = modelc['labels'][c]
    if clab == 1: 
      #print(cmid) 
      conseq = c_df.loc[c_df.id.str.contains(cmid)]
      assert conseq['id'].count() == 1 # 1-to-1
      for i in conseq.index:
        consequent = conseq['text'][i]
        if len(antecedent) > 0:
          text = antecedent + ' ' + consequent
        else:
          text = consequent  
     
  if len(text) == 0:
    slab = '0'
    for s in models.index:
      smid = models['id'][s]
      slab = models['labels'][s]
      if slab == 1:
        #print(smid)
        simple = s_df.loc[s_df.id.str.contains(smid)]
        assert simple['id'].count() == 1 # 1-to-1
        for i in simple.index:
          stext = simple['text'][i]
          text = stext                
      
  minis = 1000
  if len(text) == 0 or text == consequent: # guess antesedent
    #print('guessing...')
    for s in models.index:
      smid = models['id'][s]
      simple = s_df.loc[s_df.id.str.contains(smid)]
      assert simple['id'].count() == 1 # 1-to-1    
      for i in simple.index:
        sid = simple['id'][i]
        stext = simple['text'][i]
        stext = stext.lstrip() 
        stext = stext.rstrip()
        stext = stext.replace('   ',' ')
        score = get_score(stext)
        if score < minis :
          minis = score
          if '|' in stext:
            texts = stext.split('|')
            if len(texts) > 0:
              antecedent = texts[0]
              if len(consequent) == 0 and len(texts) > 1 :
                consequent = texts[1]
              
                          
  if len(antecedent) == 0 and '|' in text:
    antecedent =  text.split('|')[0]

  if len(consequent) == 0 and '|' in text and len(text.split('|')) > 1:
    consequent =  text.split('|')[1]

  antecedent = antecedent.lstrip()
  antecedent = antecedent.rstrip()
  consequent = consequent.lstrip()
  consequent = consequent.rstrip()
  antecedent = antecedent.replace(' \'s','\'s')
  antecedent = antecedent.replace(' \'d','\'d')
  consequent = consequent.replace(' \'s','\'s')
  consequent = consequent.replace(' \'d','\'d')
  consequent = consequent.replace('$ ','$')
  antecedent = antecedent.replace('$ ','$')
  consequent = consequent.replace(' %','%')
  antecedent = antecedent.replace(' %','%')
  consequent = consequent.replace('/ ','/')
  antecedent = antecedent.replace('/ ','/')
  consequent = consequent.replace(' /','/')
  antecedent = antecedent.replace(' /','/')
  consequent = consequent.replace(' -','-')
  antecedent = antecedent.replace(' -','-')
  #consequent = consequent.replace('- ','-')
  #antecedent = antecedent.replace('- ','-')
  consequent = consequent.replace(' .','.')
  antecedent = antecedent.replace(' .','.')
  consequent = consequent.replace(' ;',';')
  antecedent = antecedent.replace(' ;',';')
  antecedent = antecedent.replace(' \'nt','\'nt')
  consequent = consequent.replace(' \'nt','\'nt')
  antecedent = antecedent.replace('it s','its')
  consequent = consequent.replace('it s','its')
  antecedent = antecedent.replace('   ',' ')
  consequent = consequent.replace('   ',' ')
  antecedent = antecedent.replace('  ',' ')
  consequent = consequent.replace('  ',' ')
  #print(antecedent, consequent)
  astart =-1
  aend = -1
  cstart = -1
  cend = -1

  if len(antecedent) > 0 :
    try:
      astart = original.index(antecedent[0:15])
    except:
      print(id, original, antecedent)
    aend = astart + len(antecedent)

  if len(consequent) > 0 :
    try:
      cstart = original.index(consequent[0:15])
    except:
      print(id, original, consequent)     
    cend = cstart + len(consequent)

  result_row = pd.Series( data={
    'sentenceID': id,
    'antecedent_startid':astart,
    'antecedent_endid': aend,
    'consequent_startid': cstart,
    'consequent_endid': cend
  })

  submit_df = submit_df.append(result_row, ignore_index=True)
  t= t+1

  sleep(1)

submit_df.to_csv(prefix+'subtask2.csv',index=False, header=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid'])  


In [0]:
#@title
submit_df.to_csv(prefix+'subtask2.csv',index=False)  
submit_df.head()

In [0]:
!zip submission_task2.zip subtask2.csv
!mv /content/submission_task2.zip /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/